In [1]:
import pandas as pd
import pandas_ml as pdml
import numpy as np
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.metrics import roc_auc_score as AUC

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

pd.set_option('display.max_rows', 8)
pd.set_option('display.max_rows', 6)

train_df = pd.read_csv('data3.csv',header=0, index_col=0)
test_df = pd.read_csv('quiz.csv',header=0, index_col=0)
print(train_df.shape)
# convert to pdml.ModelFrame
#train_df = pdml.ModelFrame(train_df,target='label')
#train_df=train_df.dropna()
#test_df = pdml.ModelFrame(test_df)
#test_df=test_df.dropna()
#print (train_df)
print(test_df.shape)

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(126837, 52)
(31709, 51)


In [2]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

feature_columns_to_use = ([x for x in train_df.columns])
feature_columns_to_use.pop(-1)

numeric_cols = ['59','60']
nonnumeric_columns = feature_columns_to_use[0:46]+feature_columns_to_use[48:]

# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)
#print(big_X_imputed)

In [25]:
# To handle categorical features, we need to change
# them to columns of integer values.
from sklearn.preprocessing import LabelEncoder, StandardScaler
le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

# Prepare the inputs for the model
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['label']
print (train_X[0])

[  0.00000000e+00   2.50000000e+01   1.00000000e+00   7.00000000e+00
   1.00000000e+00   0.00000000e+00   2.50000000e+01   0.00000000e+00
   4.00000000e+00   7.00000000e+01   0.00000000e+00   7.90000000e+02
   2.00000000e+00   8.70000000e+01   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00
   0.00000000e+00   0.00000000e+00   2.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.00000000e+00   1.01000000e+02
   9.90000000e+01   2.08800000e+03   8.00000000e+01   4.90000000e+01
   0.00000000e+00   0.00000000e+00   0.00000000e+00]


In [5]:
from sklearn.preprocessing import LabelBinarizer
train_X = StandardScaler().fit_transform(train_X)
test_X = StandardScaler().fit_transform(test_X)

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y)
labels_train = LabelBinarizer().fit_transform(y_train)
labels_test = LabelBinarizer().fit_transform(y_test)
from sklearn.neural_network import MLPClassifier

In [17]:
#0.821885840429
#clf = MLPClassifier(random_state=1, max_iter=800, warm_start=True)
#350 - 0.810469883318
#520 - 0.90403658152

#0.894323557237
#clf = MLPClassifier(random_state=1, max_iter=800, warm_start=True, early_stopping=True, hidden_layer_sizes=(450,))

#0.9
#clf = MLPClassifier(random_state=0, max_iter=800, warm_start=True, early_stopping=True, hidden_layer_sizes=(520,))

#0.903752759382, 0.911321349732, 0.910848312835, 0.91160517187
clf = MLPClassifier(random_state=None, max_iter=800, algorithm='l-bfgs',
                    warm_start=False, shuffle=False, alpha=1e-9,
                    early_stopping=False, hidden_layer_sizes=(550,))

#0.890665405235, 0.913844213182
#clf = MLPClassifier(random_state=1, max_iter=800, warm_start=True, early_stopping=True, hidden_layer_sizes=(560,))

clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print (score)

predictions = []
predictions=clf.predict(test_X)

0.915105644907


In [18]:
indx=[x for x in range(1,31710)]
submission = pd.DataFrame({'Id': indx,
                           'Prediction': predictions })
submission.to_csv("submission.csv", index=False, sep=",")